<a href="https://colab.research.google.com/github/ToanToan110/A-B-Testing/blob/main/A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> Giả sử ta đã thực hiện một loạt các chiến dịch khuyến mãi trên nhiều phân khúc khách hàng và nhận thấy có sự khác biệt về số lượt mua.

Liệu sự khác biệt này có phải xảy ra do ngẫu nhiên hay không, nếu không thì yếu tố nào ảnh hưởng đến sự khác biệt này?

=> Thực hiện kiểm định A/B Testing để kiểm tra giả thuyết trên

---



In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
import plotly.express as px

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Driven Growth Series/ConversionCustomer.csv")
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [ ]:
df = df.sample(frac=0.25)
df.describe()

,recency,history,used_discount,used_bogo,is_referral,conversion
count,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000
mean,5.775938,241.727315,0.546937,0.550562,0.503500,0.145187
std,3.503819,254.303336,0.497808,0.497452,0.500003,0.352301
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,65.360000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,157.610000,1.000000,1.000000,1.000000,0.000000
75%,9.000000,324.937500,1.000000,1.000000,1.000000,0.000000
max,12.000000,2895.110000,1.000000,1.000000,1.000000,1.000000


# Có sự khác biệt về tỉ lệ chuyển đổi giữa nhóm đươc áp dụng khuyến mãi và nhóm không áp dụng khuyến mãi hay không?

In [ ]:
# Filter data for test and control groups
test_results = df[(df.offer == 'Discount') | (df.offer == 'By one get One')].conversion
control_results = df[df.offer == 'No Offer'].conversion
# Create a DataFrame with the filtered data
data = pd.DataFrame({'test': test_results, 'control': control_results})
# Plot using Plotly Express
px.histogram(data, histnorm='probability density', barmode='group',
                   title='Mean Conversion by Group', labels={'value': 'conversion'})

**Nhận xét**: Có vẻ như tỉ lệ chuyển đổi ở nhóm được áp dụng khuyến mãi cao hơn so với nhóm không áp được áp dụng khuyển mãi

In [ ]:
# @title Xác định giả thuyết kiểm định:
# > Kiểm định 1: Gọi H0 là giả thuyết "Tập kiểm soát và tập thử nghiệm có số lượng đặt hàng như nhau".

In [ ]:
#@title Tiến hành kiểm định và đánh giá
from scipy import stats
test_result = stats.ttest_ind(test_results, control_results)
print(test_result)

TtestResult(statistic=11.801956190103459, pvalue=6.01817452752217e-32, df=10690.0)


**Nhận xét**:

- Xác suất thấp nhất để giả thuyết H0 xảy ra là 1.268 * 10^(-112).. Quá nhỏ

- p-value nhỏ hơn mức ý nghĩa Alpha (0.05) => Bác bỏ giả thuyết H0

Như vậy có sự khác nhau giữa những khách hàng được áp dụng khuyến mãi và những khách hàng không được áp dụng khuyến mãi.

# Có sự khác biệt về tỉ lệ chuyển đổi giữa 3 nhóm Discount, Buy One get One và No Discount hay không?

In [ ]:
# Filter data for test and control groups
bogo_df = df[df.offer == 'Buy One Get One'].conversion
dis_df = df[df.offer == 'Discount'].conversion
no_offer_df = df[df.offer == 'No Offer'].conversion
# Create a DataFrame with the filtered data
data = pd.DataFrame({'BOGO': bogo_df, 'Discount': dis_df, 'No Offer': no_offer_df})
# Plot using Plotly Express
px.histogram(data, histnorm='probability density', barmode='group',
                   title='Mean Conversion by Group', labels={'value': 'conversion'})

**Nhận xét:**

Cũng có khác biệt giữa các nhóm khuyến mãi với nhau


Do đây là kiểm định với 3 biến quan sát => áp dụng kiểm định anova

In [ ]:
#@title Tiến hành kiểm định và đánh giá
from scipy import stats
avova_result = stats.f_oneway(bogo_df, dis_df, no_offer_df)
print(avova_result)

F_onewayResult(statistic=69.0539855540847, pvalue=1.3770545305149383e-30)


Tương tự ta thấy p-value rất nhỏ, chứng tỏ kết quả kiểm định đang bác bỏ giả thuyết H0

=> Có ít nhất một nhóm có tỉ lệ chuyển đổi khác biệt so với phần còn lại

# Tác động của kênh bán hàng (channel) lên tỉ lệ chuyển đổi bị ảnh hưởng như thế nào theo chương trình khuyến mãi (offer)?

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
model = smf.ols(formula='conversion ~ offer + channel + offer:channel', data=df).fit()
aov_table = anova_lm(model, typ=2)
print(np.round(aov_table))

               sum_sq       df     F  PR(>F)
offer            17.0      2.0  69.0     0.0
channel           4.0      2.0  15.0     0.0
offer:channel     0.0      4.0   1.0     1.0
Residual       1982.0  15991.0   NaN     NaN


=> Giá trị PR(>F) ~ 0, như vậy sự tương tác giữa channel và offer là không có ý nghĩa

# Vậy cỡ mẫu là bao nhiêu là đủ lớn để đạt được ý nghĩa thống kê

In [ ]:
from statsmodels.stats import power
ss_analysis = power.TTestIndPower()
def calculate_sample_size(c_data, column_name, target,ratio):
    value_mean = c_data[column_name].mean()
    value_std = c_data[column_name].std()

    value_target = value_mean * target

    effect_size = (value_target - value_mean)/value_std

    power = 0.8
    alpha = 0.05
    ss_result = ss_analysis.solve_power(effect_size=effect_size, power=power,alpha=alpha, ratio=ratio , nobs1=None)
    print(int(ss_result))
calculate_sample_size(df,'conversion', 1.05, 1)

36972
